In [1]:
from __future__ import division
import imp
import uproot
import matplotlib
#matplotlib.use('agg')
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import pandas as pd
import os
import ROOT
import time
import math
import collections
from array import array

inputdir = '/home/tmettler/Desktop/v08_00_00_33/V08_00_00_35/weighted/'
outputdir = inputdir+'xsec_all_genie_theta/' 
output_filedir = outputdir
input_filedir = '/home/tmettler/Desktop/v08_00_00_33/V08_00_00_35/weighted/'
lib_function_dir = '/home/tmettler/Desktop/uBoone/do_plots/'

# helper functions
globale = imp.load_source('globale',lib_function_dir+'globale.py')
NuCC = imp.load_source('NuCC_function',lib_function_dir+'NuCC_function.py')
NuCC_w = imp.load_source('NuCCWeight_function',lib_function_dir+'NuCCWeight_function.py')


Welcome to JupyROOT 6.10/08


In [2]:
#!jupyter nbconvert --to script numuCCana_muon_cut_weight.ipynb

In [3]:
# initialte ROOT default canvas
ROOT.gStyle.SetOptStat(0)
c1 = ROOT.TCanvas("c1","c1",1600,1200)
c1.SetGrid(1)
c1.SetLeftMargin(0.14)
c1.SetRightMargin(0.18)
c1.SetBottomMargin(0.14)

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1


# All Genie cross section variation 100 multisims, cos(Theta)

In [4]:
# Load input files
outputdir_png, outputdir_root, outputdir_pdf = NuCC.prepareOutput2(outputdir)
try:
    os.stat(output_filedir)
except:
    os.mkdir(output_filedir)
RootFile = ROOT.TFile(output_filedir+"All_Genie_theta.root","RECREATE");

#filename_overlay = 'NuCCana_overlay_V26_mar18.root'
filename_overlay = 'NuCCana_overlay_V26_weight.root'
filename_data = 'NuCCana_data_V25.root'
filename_ext = 'NuCCana_ext_V25.root'
filename_dirt = 'NuCCana_dirt_V26_weight.root'
    
tree_name = 'numuCCAna'

In [5]:
#Open all the trees of the four files (data, ext, dirt, overlay)

data, ext, dirt, overlay = NuCC.openTrees(inputdir, filename_data, filename_ext, filename_dirt, filename_overlay, tree_name)
NuCC.printNumberOfEntries(data,ext,dirt,overlay)

pot_overlay = NuCC.getPOT(inputdir,filename_overlay,tree_name)
pot_dirt =  NuCC.getPOT(inputdir,filename_dirt,tree_name)
#V25 files
pot_data =    7.644e+18  # best with tor875
data_trigger = 1838700.0 #2220362.0 #1854495.0 #4743794 # 1987072.0 # E1DCNT_wcut
ext_trigger =  18997529.0  #2120135 #5685315 # EXT

print 'POT: '
print 'Data:\t\t', pot_data
print 'Ext:\t\t', 0
print 'Overlay:\t', pot_overlay
print 'Dirt:\t\t', pot_dirt
print ''
sample = [data,ext,overlay,dirt]
scale = {data:1.0,ext:1.0,overlay:1.0,dirt:1.0}
name = {data:'data',ext:'ext',overlay:'overlay',dirt:'dirt'}

scale[data], scale[ext], scale[dirt], scale[overlay] = NuCC.calculateScale(data_trigger, ext_trigger, pot_data, pot_dirt, pot_overlay)

scale[dirt] = scale[dirt]
scale[overlay] = scale[overlay]
print 'Scalefactors: '
print 'Data:\t\t', scale[data]
print 'Ext:\t\t', scale[ext]
print 'Overlay:\t', scale[overlay]
print 'Dirt:\t\t', scale[dirt]


Overlay: Number of Entries:	989745
Data: Number of Entries:	28005
Ext: Number of Entries:		222702
Dirt: Number of Entries:	40808

POT: 
Data:		7.644e+18
Ext:		0
Overlay:	1.24025430537e+21
Dirt:		1.25019694173e+20

Scalefactors: 
Data:		1.0
Ext:		0.0967862715198
Overlay:	0.00616325213863
Dirt:		0.0611423668132


Warning in <TClass::Init>: no dictionary for class ROOT::TIOFeatures is available


In [6]:
if 1:
    #filename_overlay = 'NuCCana_overlay_V26_mar18_noflux.rootout4.root'
    filename_overlay = filename_overlay+'out4wG.root'
    #filename_overlay = 'NuCCana_overlay_points_1kev.rootout2.root'
    filename_data = filename_data+'out4.root'
    filename_ext = filename_ext+'out4.root'
    filename_dirt = filename_dirt+'out4.root'

    tree_name = 't_out'

    data_out, ext_out, dirt_out, overlay_out = NuCC.openTreesOut(inputdir, filename_data, filename_ext, filename_dirt, filename_overlay, tree_name)
    NuCC.printNumberOfEntries(data_out,ext_out,dirt_out,overlay_out)

    sample_out = [data_out,ext_out,overlay_out,dirt_out]
    scale_out = {data_out:1.0,ext_out:1.0,overlay_out:1.0,dirt_out:1.0}
    name_out = {data_out:'data',ext_out:'ext',overlay_out:'overlay',dirt_out:'dirt'}

    scale_out[data_out], scale_out[ext_out], scale_out[dirt_out], scale_out[overlay_out] = NuCC.calculateScale(data_trigger, ext_trigger, pot_data, pot_dirt, pot_overlay)
    scale_out[dirt_out] = scale_out[dirt_out]
    scale_out[overlay_out] = scale_out[overlay_out]

Overlay: Number of Entries:	989714
Data: Number of Entries:	28004
Ext: Number of Entries:		222701
Dirt: Number of Entries:	40800



In [7]:
##### flux and number of tragets parameters###
flux = 1.16859e11/1.592e20 # flux per POT per cm2
print flux
flux = 7.3789785277e-10
print flux
roh_data = 1.3836 #data denisity g/cm3
roh_MC = 1.3954 # MC denisity g/cm3
mol = 39.95 # g for argon
N_A = 6.022140857e23 # molec/mol avogadro zahl
N_nucleons = 40.0
V_fid = ((254.8-10)-(-1.55+10))*((117.47-10)-(-115.53+10))*((1036.9-50)-(-0.1+10))
print 'Fiducial Volume: ', V_fid
##############################################

beam_flux = flux * pot_data
print 'Beam flux = {:.5e}'.format(beam_flux),' /cm2'
N_tot = roh_data*N_A*N_nucleons*V_fid/mol
print 'Number of target nuclei= {:.5e}'.format(N_tot),' /cm3'

7.34038944724e-10
7.3789785277e-10
Fiducial Volume:  49184671.35
Beam flux = 5.64049e+09  /cm2
Number of target nuclei= 4.10331e+31  /cm3


In [8]:
# Define signals

fidVol = '(Nu_Vx_sce>(-1.55+10) && Nu_Vx_sce<(254.8-10)) && (Nu_Vy_sce>(-115.53+10) && Nu_Vy_sce<(117.47-10)) &&(Nu_Vz_sce>(-0.1+10) && Nu_Vz_sce<(1036.9-50))'
MCfidVol = '(MCNu_Vx>(-1.55+10) && MCNu_Vx<(254.8-10)) && (MCNu_Vy>(-115.53+10) && MCNu_Vy<(117.47-10)) &&(MCNu_Vz>(-0.1+10) && MCNu_Vz<(1036.9-50))'
numu_signal = 'fidVol && MCfidVol && MCNu_CCNC==0 && MCNu_PDG==14 && MCTrackPDG==13 && MCTrackPurity>0.5' # numu CC signal definition
numu_true = 'MCfidVol && MCNu_CCNC==0 && MCNu_PDG==14' # numu CC signal definition
numu_nomu = 'fidVol && MCfidVol && MCNu_CCNC==0 && MCNu_PDG==14 && MCTrackPDG!=13 && MCTrackPurity>0.5' # not an MC muon
numu_lowpur = 'fidVol && MCfidVol && MCNu_CCNC==0 && MCNu_PDG==14 && MCTrackPurity<0.5' #low purity
numu_nc = 'fidVol && MCfidVol && MCNu_CCNC==1' # nutral current
numu_ov = 'fidVol && !MCfidVol' # out of fiducial
numu_other = 'fidVol && MCfidVol && MCNu_CCNC==0 && MCNu_PDG!=14' # e.g anti nu or nue
#signal = 'MCfidVol && MCNu_CCNC==0 && MCNu_PDG==14'
for x in sample:
    x.SetAlias('muon','(muon_candidate_key==track_key)')
    
num_fidVol = {}
for x in sample:
    x.SetAlias('fidVol',fidVol)
    x.SetAlias('MCfidVol',MCfidVol)
    x.SetAlias('numu_signal',numu_signal)
    x.SetAlias('numu_true',numu_true)
    x.SetAlias('numu_nomu',numu_nomu)
    x.SetAlias('numu_lowpur',numu_lowpur)
    x.SetAlias('numu_nc',numu_nc)
    x.SetAlias('numu_ov',numu_ov)
    x.SetAlias('numu_other',numu_other)
    num_fidVol[x] = x.GetEntries('fidVol && muon')*scale[x]
    
tot_num_fidVol = num_fidVol[ext]+num_fidVol[dirt]+num_fidVol[overlay]
overlay_signals = {'numu_signal','numu_nomu','numu_lowpur','numu_nc','numu_ov','numu_other'}

In [9]:
for x in sample_out:
    x.SetAlias('muon','(track_key == key_muon)')

num_fidVol = {}
for x in sample_out:
    x.SetAlias('fidVol',fidVol)
    x.SetAlias('MCfidVol',MCfidVol)
    x.SetAlias('numu_signal',numu_signal)
    x.SetAlias('numu_true',numu_true)
    x.SetAlias('numu_nomu',numu_nomu)
    x.SetAlias('numu_lowpur',numu_lowpur)
    x.SetAlias('numu_nc',numu_nc)
    x.SetAlias('numu_ov',numu_ov)
    x.SetAlias('numu_other',numu_other)

In [10]:
# Load the global variables for access of functions
NuCC.loadGlobal(data,ext,dirt,overlay,data_out,ext_out,dirt_out,overlay_out,scale,scale_out,tot_num_fidVol,overlay_signals,sample,sample_out, name,name_out, outputdir_png, outputdir_root,outputdir_pdf)
#NuCC.printGlobal()

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1


In [11]:
# initialte ROOT default canvas
ROOT.gStyle.SetOptStat(0)
c1 = ROOT.TCanvas("c1","c1",1600,1200)
c1.SetGrid(1)
c1.SetLeftMargin(0.14)
c1.SetRightMargin(0.18)
c1.SetBottomMargin(0.14)

In [12]:
'''track_start_border_x = '(TrackStart_x_sce <(-1.55+1) || TrackStart_x_sce > (254.8-1))'
track_end_border_x = '(TrackEnd_x_sce <(-1.55+1) || TrackEnd_x_sce > (254.8-1))'
track_start_border_y = '(TrackStart_y_sce <(-115.53+1) || TrackStart_y_sce > (117.47-1))'
track_end_border_y = '(TrackEnd_y_sce <(-115.53+1) || TrackEnd_y_sce > (117.47-1))'
track_start_border_z = '(TrackStart_z_sce <(0.1+1) || TrackStart_z_sce > (1036.9-1))'
track_end_border_z = '(TrackEnd_z_sce <(0.1+1) || TrackEnd_z_sce > (1039.9-1))'
'''
track_start_border_x = '(TrackStart_x_sce <(-1.55+5) || TrackStart_x_sce > (254.8-5))'
track_end_border_x = '(TrackEnd_x_sce <(-1.55+5) || TrackEnd_x_sce > (254.8-5))'
track_start_border_y = '(TrackStart_y_sce <(-115.53+5) || TrackStart_y_sce > (117.47-5))'
track_end_border_y = '(TrackEnd_y_sce <(-115.53+5) || TrackEnd_y_sce > (117.47-5))'
track_start_border_z = '(TrackStart_z_sce <(0.1+5) || TrackStart_z_sce > (1036.9-5))'
track_end_border_z = '(TrackEnd_z_sce <(0.1+5) || TrackEnd_z_sce > (1039.9-5))'

track_end_uncontained = '(' + track_end_border_x + ' || ' + track_end_border_y + ' || ' + track_end_border_z+ ')'


data.SetAlias("track_end_uncontained",track_end_uncontained)
ext.SetAlias("track_end_uncontained",track_end_uncontained)
overlay.SetAlias("track_end_uncontained",track_end_uncontained)
dirt.SetAlias("track_end_uncontained",track_end_uncontained)
data_out.SetAlias("track_end_uncontained",track_end_uncontained)
ext_out.SetAlias("track_end_uncontained",track_end_uncontained)
overlay_out.SetAlias("track_end_uncontained",track_end_uncontained)
dirt_out.SetAlias("track_end_uncontained",track_end_uncontained)

data.SetAlias("crt_cut","(abs(crtt0_time+(crt_trig_corr_med)/1000-4)<0.9 || crtt0_time==-1)")
ext.SetAlias("crt_cut","(abs(crtt0_time+(crt_trig_corr_med)/1000-3.57+3.195-4)<0.9 || crtt0_time==-1)")
overlay.SetAlias("crt_cut","(abs(crtt0_time-3.95)<0.9 || crtt0_time==-1)")
dirt.SetAlias("crt_cut","(abs(crtt0_time-3.95)<0.9 || crtt0_time==-1)")
data_out.SetAlias("crt_cut","(abs(crtt0_time+(crt_trig_corr_med)/1000-3.95)<0.9 || crtt0_time==-1)")
ext_out.SetAlias("crt_cut","(abs(crtt0_time+(crt_trig_corr_med)/1000-3.57+3.195-3.95)<0.9 || crtt0_time==-1)")
overlay_out.SetAlias("crt_cut","(abs(crtt0_time-3.95)<0.9 || crtt0_time==-1)")
dirt_out.SetAlias("crt_cut","(abs(crtt0_time-3.95)<0.9 || crtt0_time==-1)")

crt_tom_cut = 'nr_crthit_top==0 && crthit_vertex_zcut==0 && (track_end_uncontained==1 || nr_crthit_beam_tres==0) && crt_cut'

data.SetAlias("crt_tom_cut",crt_tom_cut)
ext.SetAlias("crt_tom_cut",crt_tom_cut)
overlay.SetAlias("crt_tom_cut",crt_tom_cut)
dirt.SetAlias("crt_tom_cut",crt_tom_cut)
data_out.SetAlias("crt_tom_cut",crt_tom_cut)
ext_out.SetAlias("crt_tom_cut",crt_tom_cut)
overlay_out.SetAlias("crt_tom_cut",crt_tom_cut)
dirt_out.SetAlias("crt_tom_cut",crt_tom_cut)

weight_name = 'EventWeight*TunedCentralValue_Genie'


In [13]:
weight_name = 'EventWeight*TunedCentralValue_Genie'
num_universes = 100
eff_uni = np.zeros((num_universes))
mom_bins = [ -1.00, -0.50, 0.00, 0.28, 0.47, 0.63, 0.765, 0.865, 0.935, 1.00 ]
binnum = len(mom_bins) - 1

In [14]:
variable = 'cos(TrackTheta)'
cut = 'fidVol && muon && crt_tom_cut && TrackScore>0.8\
                && TrackLength>20 && TrackPID_chiproton>78 && NuScore>0.1'

h_ext_mom = ROOT.TH1F('h_ext_mom','h_ext_mom',binnum,array('f',mom_bins))
h_data_mom = ROOT.TH1F('h_data_mom','h_data_mom',binnum,array('f',mom_bins))
h_dirt_mom = ROOT.TH1F('h_dirt_mom','h_dirt_mom',binnum,array('f',mom_bins))

globale.ext_out.Draw(variable+'>>h_ext_mom','('+cut+')')
globale.data_out.Draw(variable+'>>h_data_mom','('+cut+')')
globale.dirt_out.Draw(variable+'>>h_dirt_mom',weight_name+'*('+cut+')')

h_data_mom.Scale(globale.scale[globale.data])
h_ext_mom.Scale(globale.scale[globale.ext])
h_dirt_mom.Scale(globale.scale[globale.dirt])
#h_overlay.Scale(globale.scale[globale.overlay])

bkg_ext2 = h_ext_mom.GetSumOfWeights()
num_data2 = h_data_mom.GetSumOfWeights()
bkg_dirt2 = h_dirt_mom.GetSumOfWeights()

bkg_ext = h_ext_mom.Integral(0,binnum+1)
num_data = h_data_mom.Integral(0,binnum+1)
bkg_dirt = h_dirt_mom.Integral(0,binnum+1)

bkg_ext3 = ext_out.GetEntries(cut)
num_data3 = data_out.GetEntries(cut)
bkg_dirt3 = dirt_out.GetEntries(cut)

h_data_mom.Write('h_data_mom')
h_ext_mom.Write('h_ext_mom')
h_dirt_mom.Write('h_dirt_mom')

print 'Data: ', num_data,' Ext: ', bkg_ext,' Dirt: ',bkg_dirt
print 'Data: ', num_data2,' Ext: ', bkg_ext2,' Dirt: ',bkg_dirt2
print 'Data: ', num_data3,' Ext: ', bkg_ext3,' Dirt: ',bkg_dirt3


Data:  1348.0  Ext:  142.179031849  Dirt:  32.1298258305
Data:  1348.0  Ext:  142.179031849  Dirt:  32.1298258305
Data:  1348  Ext:  1469  Dirt:  451


In [15]:

variable = 'cos(MCNu_leptonTheta)'
cut = 'fidVol && muon && crt_tom_cut && TrackScore>0.8\
                && TrackLength>20 && TrackPID_chiproton>78 && NuScore>0.1'

h_1d = {}
h_1d_true = {}
for uni in range(num_universes):
    #print 'At universe: ',uni
    h_1d[uni] = ROOT.TH1F("h_1d["+str(uni)+']',"Track true momentum",binnum,array('f',mom_bins))
    h_1d_true[uni] = ROOT.TH1F("h_1d_true["+str(uni)+']',"Track true momentum",binnum,array('f',mom_bins))
    
for uni in range(num_universes):
    globale.overlay_out.Draw(variable+">>h_1d["+str(uni)+']',weight_name+'*All_Genie['+str(uni)+']*('+cut+'&& numu_signal)')
    globale.overlay_out.Draw(variable+">>h_1d_true["+str(uni)+']',weight_name+'*All_Genie['+str(uni)+']*(numu_true)')
    #globale.overlay_out.Draw(variable+">>h_1d["+str(uni)+']',weight_name+'*All_UBGenie['+str(uni)+']*('+cut+'&& numu_signal)')
    #globale.overlay_out.Draw(variable+">>h_1d_true["+str(uni)+']',weight_name+'*All_UBGenie['+str(uni)+']*(numu_true)')
    #print variable+">>h_1d_true["+str(uni)+']',weight_name+'*All_Genie['+str(uni)+']*(numu_true)'
    num_signal = h_1d[uni].Integral(0,binnum+1)
    num_true = h_1d_true[uni].Integral(0,binnum+1)
    eff_uni[uni] = num_signal/(num_true+1e-80)
    h_1d[uni].Write('h_1d['+str(uni)+']')
    h_1d_true[uni].Write('h_1d_true['+str(uni)+']')
    print 'Signal: ', num_signal,' True: ', num_true,' Eff: ',eff_uni[uni]

Signal:  214667.958008  True:  417402.386719  Eff:  0.5142949940831507
Signal:  165021.23291  True:  318245.445312  Eff:  0.5185344687277904
Signal:  155261.197754  True:  297086.03125  Eff:  0.5226135914254846
Signal:  191604.802734  True:  369643.962891  Eff:  0.5183496065674134
Signal:  165773.845215  True:  319265.308594  Eff:  0.5192353843423155
Signal:  145632.770996  True:  274995.658203  Eff:  0.5295820739414089
Signal:  159897.574707  True:  301784.455078  Eff:  0.5298403281429372
Signal:  152923.20166  True:  290608.132812  Eff:  0.5262179009932324
Signal:  151583.421387  True:  290261.638672  Eff:  0.5222302956749844
Signal:  221317.208984  True:  427933.042969  Eff:  0.5171771907329362
Signal:  165121.384277  True:  315564.470703  Eff:  0.5232572092461123
Signal:  193495.513184  True:  372931.667969  Eff:  0.5188497781309572
Signal:  187663.816895  True:  362911.667969  Eff:  0.5171060438615901
Signal:  221894.95752  True:  429806.320312  Eff:  0.516267320960282
Signal:  23

In [16]:
variable = 'cos(TrackTheta)'
cut = 'fidVol && muon && crt_tom_cut && TrackScore>0.8\
                && TrackLength>20 && TrackPID_chiproton>78 && NuScore>0.1'

bkg_uni = np.zeros((num_universes))
h_1d_bkg = {}
for uni in range(num_universes):
    #print 'At universe: ',uni
    h_1d_bkg[uni] = ROOT.TH1F("h_1d_bkg["+str(uni)+']',"Track momentum",binnum,array('f',mom_bins))

for uni in range(num_universes):
    globale.overlay_out.Draw(variable+">>h_1d_bkg["+str(uni)+']',weight_name+'*All_Genie['+str(uni)+']*('+cut+'&& !numu_signal)')
    #globale.overlay_out.Draw(variable+">>h_1d_bkg["+str(uni)+']',weight_name+'*All_UBGenie['+str(uni)+']*('+cut+'&& !numu_signal)')
    h_1d_bkg[uni].Scale(globale.scale[globale.overlay])
    bkg_uni[uni] = h_1d_bkg[uni].Integral(0,binnum+1)
    h_1d_bkg[uni].Write('h_1d_bkg['+str(uni)+']')
    #print 'Overlay: ',bkg_uni[uni]


In [17]:
# calculate total flux integrated cross section:
cut = 'fidVol && muon && TrackLength>8 && crt_tom_cut && TrackScore>0.8\
                && TrackLength>20 && TrackPID_chiproton>78 && NuScore>0.1'
weight_name = 'EventWeight*TunedCentralValue_Genie'
N_data = data_out.GetEntries(cut)
N_ext  = ext_out.GetEntries(cut)*scale_out[ext_out]
N_dirt = dirt_out.GetEntries(cut)*scale_out[dirt_out]
N_overlay = overlay_out.GetEntries(cut+'&& !numu_signal')*scale_out[overlay_out]
N_signal = overlay_out.GetEntries(cut+'&& numu_signal')*scale_out[overlay_out]
N_true = overlay_out.GetEntries('numu_true')*scale_out[overlay_out]

print N_overlay

h_weight_func = ROOT.TH1F("h_weight_func",'h_weight_func',10000,0,1000)
overlay_out.Draw(weight_name+'>>h_weight_func',cut+' && 1','0')
dirt_pass_weight = h_weight_func.GetMean() 
overlay_out.Draw(weight_name+'>>h_weight_func',cut+' && numu_signal','0') # weights for signal definition
overlay_signal = h_weight_func.GetMean()
overlay_out.Draw(weight_name+'>>h_weight_func',cut+' && !numu_signal','0') # weights for signal definition
overlay_pass_weight = h_weight_func.GetMean()

overlay_out.Draw(weight_name+'>>h_weight_func',numu_true,'0') # weights for signal definition
overlay_true_weight = h_weight_func.GetMean()

print overlay_pass_weight

N_dirt = N_dirt*dirt_pass_weight
N_overlay = N_overlay*overlay_pass_weight
N_signal = N_signal*overlay_signal
N_true = N_true*overlay_true_weight
eff = N_signal/N_true

sigma_tot = (N_data- (N_ext+N_dirt+N_overlay))/(eff*beam_flux*N_tot)
print 'Number of data:', N_data
print 'Number of ext:', N_ext
print 'Number of dirt:', N_dirt
print 'Number of overlay:', N_overlay
print 'Number of overlay signal:', N_signal
print 'Efficiency: ',eff
print 'Total integrated cross section:', sigma_tot
xsec_CV = sigma_tot

sigma_tot_MC = (N_signal)/(eff*beam_flux*N_tot)
print 'Total integrated cross section for MC:', sigma_tot_MC

179.048637879
1.11470866893
Number of data: 1348
Number of ext: 142.179032863
Number of dirt: 32.2682384335
Number of overlay: 199.587068804
Number of overlay signal: 975.196963731
Efficiency:  0.525802793429
Total integrated cross section: 8.00330534507e-39
Total integrated cross section for MC: 8.01342325881e-39


In [18]:
# calculate the total cross section and the difference to the CV
# from momentum CV xsec:
#xsec_CV = 8.00835474027e-39 # 7.6770041506e-39
xsec_uni = np.zeros((num_universes))
xsec_diff = np.zeros((num_universes))
sigma_tot = 0.0
for uni in range(num_universes):
    signal = num_data - bkg_ext - bkg_dirt - bkg_uni[uni]
    xsec_uni[uni] = (num_data - bkg_ext - bkg_dirt - bkg_uni[uni])/(eff_uni[uni] * N_tot * beam_flux)
    xsec_diff[uni] = xsec_uni[uni] - xsec_CV
    print 'Total flux integrated xsec: ',xsec_uni[uni],' change: ', (xsec_uni[uni] - xsec_CV)/xsec_CV*100
    sigma_tot = sigma_tot+xsec_diff[uni]*xsec_diff[uni]
sigma_tot = sigma_tot/num_universes
print 'xsec CV: ',xsec_CV,' deviatons: ',math.sqrt(sigma_tot),' in percent: ',math.sqrt(sigma_tot)*100.0/xsec_CV,'%'

Total flux integrated xsec:  7.902106125002904e-39  change:  -1.2644678129914817
Total flux integrated xsec:  8.020094748869643e-39  change:  0.20978087276518587
Total flux integrated xsec:  7.966645703288632e-39  change:  -0.4580562679726315
Total flux integrated xsec:  7.991689124343607e-39  change:  -0.14514279066276106
Total flux integrated xsec:  8.129493535049171e-39  change:  1.5767009321999121
Total flux integrated xsec:  8.201029709572888e-39  change:  2.4705338104845356
Total flux integrated xsec:  7.940182958427786e-39  change:  -0.7887039656411668
Total flux integrated xsec:  8.215675463478095e-39  change:  2.653530126053521
Total flux integrated xsec:  8.235843058239809e-39  change:  2.9055209459982434
Total flux integrated xsec:  7.479828450007454e-39  change:  -6.540758755155086
Total flux integrated xsec:  8.007056274678112e-39  change:  0.046867256085948834
Total flux integrated xsec:  7.741602408626406e-39  change:  -3.26993567228581
Total flux integrated xsec:  7.758

In [19]:
#get mitigation matrix for each universe for momentum
cut = 'fidVol && muon && crt_tom_cut && TrackScore>0.8\
                && TrackLength>20 && TrackPID_chiproton>78 && NuScore>0.1'

migration_matrix = np.zeros((num_universes,9,9))

h_migration = {}
h_mitigation_matrix = {}

for uni in range(num_universes):
    h_migration[uni] = ROOT.TH2F("h_migration["+str(uni)+']',"Track momentum",binnum,array('f',mom_bins),binnum,array('f',mom_bins))
    #h_migration[uni].Write('h_migration['+str(uni)+'])')
    h_mitigation_matrix[uni] = ROOT.TH2F("h_mitigation_matrix["+str(uni)+']','Migration matrix',7,0,7,6,0,6)

for uni in range(num_universes):
    overlay_out.Draw('cos(MCNu_leptonTheta):cos(TrackTheta)'+'>>h_migration['+str(uni)+']',weight_name+'*All_Genie['+str(uni)+']*('+cut+' && numu_true)','')
    #overlay_out.Draw('cos(MCNu_leptonTheta):cos(TrackTheta)'+'>>h_migration['+str(uni)+']',weight_name+'*All_UBGenie['+str(uni)+']*('+cut+' && numu_true)','')
    h_migration[uni].Write('h_migration['+str(uni)+'])')
    
mitigation_matrix = np.zeros((num_universes,9,9))
for uni in range(num_universes):
    reco_entries = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
    for j in range(1,binnum+1):
            for i in range(1,binnum+1):
                reco_entries[j-1]+=h_migration[uni].GetBinContent(i,j)
    for j in range(1,binnum+1):
        for i in range(1,binnum+1):
            mitigation_matrix[uni,i-1,j-1]= h_migration[uni].GetBinContent(i,j)/reco_entries[j-1]
            h_mitigation_matrix[uni].SetBinContent(i,j,mitigation_matrix[uni,i-1,j-1])
    h_mitigation_matrix[uni].Write('h_mitigation_matrix['+str(uni)+']')


In [ ]:
# calculate e tilde for all universes
N_sel = np.zeros((num_universes,9))
N_gen = np.zeros((num_universes,9))
eff_tilde = np.zeros((num_universes,9))
#h_eff_true = {}
for uni in range(num_universes):
    for i in range(1,binnum+1):
        N_sel[uni][i-1] = h_1d[uni].GetBinContent(i)
        N_gen[uni][i-1] = h_1d_true[uni].GetBinContent(i)
    #print N_sel[uni]
    #print N_gen[uni]
    #print mitigation_matrix[uni]
    #print mitigation_matrix[uni].dot(N_sel[uni])/(mitigation_matrix[uni].dot(N_gen[uni]))
    eff_tilde[uni] = mitigation_matrix[uni].dot(N_sel[uni])/(mitigation_matrix[uni].dot(N_gen[uni]))
np.save(output_filedir+'eff_tilde_all_genie_theta',eff_tilde)

h_eff_true = {}
h_eff_tilde = {}
for uni in range(num_universes):
    h_eff_tilde[uni] = ROOT.TH1F("h_eff_tilde["+str(uni)+']','Efficiency tilde',binnum,array('f',mom_bins))
    h_eff_true[uni] = h_1d[uni].Clone()
    h_eff_true[uni].Sumw2()
    h_eff_true[uni].Divide(h_1d_true[uni])
    for i in range(1,binnum+1):
        h_eff_tilde[uni].SetBinContent(i,eff_tilde[uni][i-1])
        h_eff_tilde[uni].SetBinError(i,h_eff_true[uni].GetBinError(i))
    h_eff_tilde[uni].SetYTitle("reconstructed efficiency")
    h_eff_tilde[uni].SetXTitle("cos(\Theta)_{\mu}")
    h_eff_tilde[uni].Write('h_eff_tilde['+str(uni)+']')
    h_eff_true[uni].Write('h_eff_true['+str(uni)+']')

In [ ]:
h_xsec = {}
for uni in range(num_universes): 
    h_xsec[uni] = h_data_mom.Clone()
    h_xsec[uni].Sumw2()
    h_xsec[uni].Add(h_ext_mom,-1)
    h_xsec[uni].Add(h_dirt_mom,-1)
    h_xsec[uni].Add(h_1d_bkg[uni],-1)
    print h_xsec[uni].GetSumOfWeights()


In [ ]:
for uni in range(num_universes): 
    h_xsec[uni].Divide(h_eff_tilde[uni])
    h_xsec[uni].Scale(1.0/(N_tot * beam_flux))
    for i in range(1,binnum+1):
        h_xsec[uni].SetBinContent(i,h_xsec[uni].GetBinContent(i)/h_xsec[uni].GetBinWidth(i))
    h_xsec[uni].Write('h_xsec['+str(uni)+']')

In [ ]:
h_xsec[0].Draw()
c1.Draw()

In [ ]:
# close the root file with all the histos
RootFile.Close()